- This code has written by Ahmadreza Attarpour (a.attarpour@mail.utoronto.ca)
- It's a practice to check how we can use LangChain/LangGraph to build AI Assistants

-------------------------------------

In [1]:
import os

In [2]:
# load the environment variables from the .env file
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Read the variable from the environment and set it as a env variable
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
if LANGCHAIN_API_KEY:
    os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
if LANGCHAIN_PROJECT:
    os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

In [5]:
# load the LLM and embeddings from langchain_google_genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Simple AI Assistant

AI assistants are built on large language models (LLMs) that can comprehend user input and generate helpful responses. They aim to replicate human-like conversations and continuously improve by learning from each interaction.

User Input

   ↓

[ LLM (AI Assistant) ]

   ↓
   
AI Response


In [ ]:
while True:
    question=input ("type your question. if you want to quit the chat write quit")
    if question == "quit":
        print ("goodbye take care yourself")
        break
    else:
        print (llm.invoke(question).content)

Hi there! How can I help you today?
goodbye take care yourself


## In the previous version we don't have any memory, so the LLM doesn't have access to the previous history.

In [7]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [8]:
# define a dictionary to store the chat history
store = {}

In [9]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Get the chat history for a session."""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [10]:
'''
This config dictionary tells LangChain which session ID it should use for the current interaction. 
The session_id is a string identifier (here it's "firstchat"), used to fetch or create the corresponding chat history from your store.
LangChain’s invoke() method accepts a config parameter that may include different types of metadata.
"configurable" is a reserved keyword in LangChain for passing runtime settings — in this case, the session_id.
'''
config = {"configurable": {"session_id": "firstchat"}}

In [11]:
model_with_memory = RunnableWithMessageHistory(
    llm,
    get_session_history)

In [12]:
model_with_memory.invoke(("Hi, I'm Reza"), config=config).content

"Hi Reza, it's nice to meet you! How can I help you today?"

In [13]:
model_with_memory.invoke(("what's my name?"), config=config).content

'Your name is Reza. You just told me! 😊'

In [14]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi, I'm Reza", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Reza, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--feb704c7-0bbb-4581-b743-9fb59504983f-0', usage_metadata={'input_tokens': 6, 'output_tokens': 19, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}}), HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Reza. You just told me! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--2f4f3958-0d55-48da-b3a3-97a54fb380c0-0', usage_metadata={'input_tokens': 30, 'output_tokens': 